## Promt
I want to create a semiautomatic Job Post application.
Using Deepseek API write apyton code to in a jupyter notebook to:

- Load job posts: written in .txt files available on folder "posts"
- Generate questions for key words: like “You are an expert in crafting resumes optimized to make resumes stand out when being reviewed by applicant tracking systems. Attached is the job description to which I’m applying, and my most recent resume. First, highlight keywords present in the job description that are lacking in my resume. Then, note which keywords may be the most difficult to incorporate into a rewrite of my current resume bullets. Finally, ask me any questions about my career experience to help you be able to incorporate those additional keywords (as I will eventually have you rewrite my resume).”
- Question answering: Use Deepseek to generate possible answers
- Generation of general CV: Using Deepseek and previous answers, after manual improvement
- Generation of Individual CVs and cover letters: Base on general CV and job posts.  CV should be easy to scrape by Indeed and Careerbeacon (think in format of job experience to differentiate title of the job, location, start month, start year, end month or end year, including present.  Make easy to parse in education Name of University, Type of title, year start, year end, description of the education)

Example code of Deepseek API:

from openai import OpenAI

client = OpenAI(api_key="<DeepSeek API Key>", base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)

In [1]:
import os
import datetime
from openai import OpenAI  # Deepseek API example uses openai's interface.
from docx import Document  # For writing DOCX files
from credentials import APIKEY

# Initialize Deepseek API client
client = OpenAI(api_key=APIKEY, base_url="https://api.deepseek.com")

def call_deepseek(prompt):
    """
    Calls the Deepseek API with the provided prompt and returns the response text.
    """
    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[
            {"role": "system", "content": "You are a helpful expert in crafting resumes optimized to make resumes stand out when being reviewed by applicant tracking systems."},
            {"role": "user", "content": prompt},
        ],
        stream=False
    )
    return response.choices[0].message.content


In [2]:
# STEP 1: Load job posts

posts_folder = "posts"
job_posts = {}

# Loop through each .txt file in the posts folder.
for filename in os.listdir(posts_folder):
    if filename.endswith(".txt"):
        file_path = os.path.join(posts_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            job_posts[filename] = f.read()
print(f"Loaded {len(job_posts)} job post(s).")


Loaded 6 job post(s).


In [3]:
# --------------------------
# STEP 2: Generate Q&A for keywords for each job post, taking into account the general resume
# --------------------------


questions_folder = "questions"
os.makedirs(questions_folder, exist_ok=True)

# Load the general resume to include in the prompt
with open("gen_resume.txt", "r", encoding="utf-8") as f:
    general_resume = f.read()


In [ ]:

# Modified prompt template that includes the general resume context
qa_prompt_template = (
    "You are an expert in crafting resumes optimized to make resumes stand out when being reviewed by applicant tracking systems. "
    "Below is my current general resume:\n\n{general_resume}\n\n"
    "Now, attached is the job description:\n\n{job_post}\n\n"
    "Based on both, generate questions about key words present in the job description that might be lacking in my resume. "
    "Also, note which keywords may be the most difficult to incorporate into a rewrite of my current resume bullets, "
    "and then ask any clarifying questions about my career experience to help incorporate those additional keywords. At the end of each question write a possible correct answer."
)

# Iterate over each job post and generate its Q&A file.
for filename, job_text in job_posts.items():
    qa_prompt = qa_prompt_template.format(general_resume=general_resume, job_post=job_text)
    qa_response = call_deepseek(qa_prompt)
    
    # Save the Q&A to a file in the questions folder, naming it with _qa suffix.
    qa_filename = os.path.join(questions_folder, os.path.splitext(filename)[0] + "_qa.txt")
    with open(qa_filename, "w", encoding="utf-8") as f:
        f.write(qa_response)
    print(f"Generated Q&A for {filename} and saved to {qa_filename}")


Generated Q&A for project_manager_Nanoptix.txt and saved to questions/project_manager_Nanoptix_qa.txt
Generated Q&A for Senior_Plant_Accountant_IrvingTissue.txt and saved to questions/Senior_Plant_Accountant_IrvingTissue_qa.txt
Generated Q&A for Regulatory_Advisor_NBEnergyUtilitiesBoard.txt and saved to questions/Regulatory_Advisor_NBEnergyUtilitiesBoard_qa.txt
Generated Q&A for Planning_Business_Partner_ WorkSafeNB.txt and saved to questions/Planning_Business_Partner_ WorkSafeNB_qa.txt
Generated Q&A for Plant_Cost_Accountant_Irving_Personal_Care.txt and saved to questions/Plant_Cost_Accountant_Irving_Personal_Care_qa.txt


In [ ]:
# STEP 3: Wait for manual review/modification of Q&A files

input("Please review and modify the Q&A files in the 'questions' folder as needed. Then press Enter to continue...")


In [4]:
# STEP 4: Generate a general CV by rewriting and augmenting gen_resume.txt

# Read the current general resume
with open("gen_resume.txt", "r", encoding="utf-8") as f:
    general_resume = f.read()

# # Create a prompt to have Deepseek rewrite and augment the general resume.
# cv_prompt = (
#     "Please rewrite and augment the following general resume to make it more effective and modern:\n\n"
#     f"{general_resume}"
# )
# new_cv = call_deepseek(cv_prompt)

# # Save the updated general resume back to file.
# with open("gen_resume.txt", "w", encoding="utf-8") as f:
#     f.write(new_cv)
# print("General resume has been updated and saved to gen_resume.txt.")


In [ ]:

# STEP 5: Generate Individual CVs and Cover Letters (DOCX)

# Get current date string (e.g., '12Mar2025')
date_str = datetime.datetime.now().strftime("%d%b%Y")

# Loop over each job post to generate individual CV and cover letter.
for filename, job_text in job_posts.items():
    qa_filename = os.path.join(questions_folder, os.path.splitext(filename)[0] + "_qa.txt")
    
    # Read the corresponding Q&A file.
    with open(qa_filename, "r", encoding="utf-8") as f:
        qa_content = f.read()
    
    # Create a prompt that includes the general resume, job description, and Q&A responses.
    individual_prompt = (
        "Using the following general resume, job description, and the Q&A responses, re-write the contents of my resume seamlessly incorporating the important keywords you originally highlighted into my bullet points accompanied by strong action verbs.\n\n"
        "Give more space to the information that matters for the recruiter to make it look like almost every professional experience has lead me to this job, without lying. Assure my bullet points are no more than 2 lines (but ideally one line each) and that the final resume is optimized for the job description.\n\n"
        "General Resume:\n"
        f"{general_resume}\n\n"
        "Job Description:\n"
        f"{job_text}\n\n"
        "Q&A Responses:\n"
        f"{qa_content}\n\n"
        "CV should be easy to scrape by Indeed and Careerbeacon"
        "The CV should be formatted so that job experience details (job title, location, start month/year, end month/year or 'present') "
        "and education details (University Name, Degree, start year, end year, and description) are clearly separated. "
        "Provide the CV and cover letter as separate outputs, with markers like 'CV:' and 'Cover Letter:'."
    )
    
    individual_response = call_deepseek(individual_prompt)
    
    # Assume the response contains "CV:" and "Cover Letter:" markers.
    try:
        cv_text, cl_text = individual_response.split("Cover Letter:", 1)
        cv_text = cv_text.replace("CV:", "").strip()
        cl_text = cl_text.strip()
    except Exception as e:
        # Fallback: use the full response for both if splitting fails.
        cv_text = individual_response
        cl_text = individual_response

    # Remove '#' and '*' symbols from the responses.
    cv_text = cv_text.replace("#", "").replace("*", "").replace("---","")
    cl_text = cl_text.replace("#", "").replace("*", "").replace("---","")
    
    # Create a DOCX file for the CV.
    cv_doc = Document()
    cv_doc.add_paragraph(cv_text)
    cv_doc_filename = f"{os.path.splitext(filename)[0]}_CV_{date_str}.docx"
    cv_doc.save(cv_doc_filename)
    
    # Create a DOCX file for the Cover Letter.
    cl_doc = Document()
    cl_doc.add_paragraph(cl_text)
    cl_doc_filename = f"{os.path.splitext(filename)[0]}_CL_{date_str}.docx"
    cl_doc.save(cl_doc_filename)
    
    print(f"Generated CV: {cv_doc_filename} and Cover Letter: {cl_doc_filename}")

Generated CV: Technical_Product Manager_Data_Analytics_Platform_ASURION_CV_20Mar2025.docx and Cover Letter: Technical_Product Manager_Data_Analytics_Platform_ASURION_CL_20Mar2025.docx
Generated CV: Business_Analyst_JDIIT_CV_20Mar2025.docx and Cover Letter: Business_Analyst_JDIIT_CL_20Mar2025.docx
Generated CV: Project_Manager_CMDSS_CV_20Mar2025.docx and Cover Letter: Project_Manager_CMDSS_CL_20Mar2025.docx
Generated CV: Business_Analyst_MEDAVIE_CV_20Mar2025.docx and Cover Letter: Business_Analyst_MEDAVIE_CL_20Mar2025.docx
Generated CV: Manager_Data_&_Analytics_360I_CV_20Mar2025.docx and Cover Letter: Manager_Data_&_Analytics_360I_CL_20Mar2025.docx
Generated CV: Pricing_Analyst_IOIL_CV_20Mar2025.docx and Cover Letter: Pricing_Analyst_IOIL_CL_20Mar2025.docx
Generated CV: Analyst_Pricing_Performance_SYSCO_CV_20Mar2025.docx and Cover Letter: Analyst_Pricing_Performance_SYSCO_CL_20Mar2025.docx
Generated CV: Lead_Business_Analyst_IOIL_CV_20Mar2025.docx and Cover Letter: Lead_Business_Analyst

In [6]:
import os

# Define the folder path
input_folder = 'questions'
output_file = '1.txt'

# Open the output file in write mode
with open(output_file, 'w', encoding='utf-8') as outfile:
    # Loop through each file in the folder
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)

        # Only process files (ignore directories)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='utf-8') as infile:
                content = infile.read()
                outfile.write(f'--- {filename} ---\n')  # Optional: mark which file the content came from
                outfile.write(content + '\n\n')